## **Nugen Intelligence**
<img src="https://nugen.in/logo.png" alt="Nugen Logo" width="200"/>

Domain-aligned foundational models at industry leading speeds and zero-data retention! To learn more, visit [Nugen](https://docs.nugen.in/introduction)


Welcome to the Nugen Intelligence Cookbook—a comprehensive guide designed to help you harness the power of domain-aligned large language models (LLMs) with industry-leading speeds and zero-data retention. At Nugen, we specialize in building highly tailored AI models that adapt to your specific needs, enabling scalable, intelligent solutions for specialized domains. 

## **Orchestrator Workers - A Developer's Guide to Task Distribution Systems**

Think of a restaurant kitchen during dinner service. The head chef acts as the orchestrator, while the line cooks are the workers. When an order comes in, the head chef doesn't cook everything themselves. Instead, they:

Analyze the order (orchestration)
Break it down into specific tasks (task distribution)
Assign different parts to specialized cooks (parallel execution)
Ensure everything comes together properly (coordination)

**The Orchestrator (Head Chef):**

- Takes in a large task (like writing product descriptions)
- Analyzes it to determine different approaches needed
- Breaks it into smaller, focused sub-tasks
- Coordinates the execution of these tasks

**The Workers (Line Cooks):**

- Each handles a specific part of the task
- Works independently but in parallel with others
- Follows specific instructions for their part
- Returns their completed work to the orchestrator

### **PART 1: Imports and Installations** 

In [7]:
!pip install pydantic --quiet

import asyncio
import json
import requests
from pydantic import BaseModel, Field
from typing import Literal, List


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### **PART 2:  Prompts that define the behavior of our system**

In [8]:
ORCHESTRATOR_PROMPT = """
Analyze this task and break it down into 2-3 distinct approaches:
Task: {task}
Provide an Analysis:
Explain your understanding of the task and which variations would be valuable.
Focus on how each approach serves different aspects of the task.
Along with the analysis, provide 2-3 approaches to tackle the task, each with a brief description:
Formal style: Write technically and precisely, focusing on detailed specifications
Conversational style: Write in a friendly and engaging way that connects with the reader
Hybrid style: Tell a story that includes technical details, combining emotional elements with specifications
Return only JSON output.
"""

WORKER_PROMPT = """
Generate content based on:
Task: {original_task}
Style: {task_type}
Guidelines: {task_description}
Return only your response:
[Your content here, maintaining the specified style and fully addressing requirements.]
"""

We have two important prompts that define the behavior of our system:

**ORCHESTRATOR_PROMPT**

1. This is a template that tells the orchestrator (the coordinator) how to analyze and break down a task into different approaches. It asks for:

    - Analysis of the given task
    - 2-3 different writing styles (formal, conversational, hybrid)
    - JSON output format


**WORKER_PROMPT** 

2. This template is used by worker models to actually generate content based on:

    - The original task
    - The specific style assigned
    - Guidelines for that approach

### **PART 3: Data Models**

In [9]:
class Task(BaseModel):
    type: Literal["formal", "conversational", "hybrid"]
    description: str

class TaskList(BaseModel):
    analysis: str 
    tasks: List[Task] = Field(..., default_factory=list)

**These Pydantic models ensure data validation and structure.**

1. The Task class represents a single approach with its type and description 

2. TaskList holds the overall analysis and list of tasks.

### **PART 4: Let's examine each function**

1. **run_nugen_llm**

In [10]:
async def run_nugen_llm(prompt: str, max_tokens: int = 1000, model: str = "nugen-flash-instruct", temperature: float = 0.1):
    """Run a single LLM call using the Nugen API."""
    url = "https://api.nugen.in/inference/completions"
    payload = {
        "max_tokens": str(max_tokens),
        "model": model,
        "prompt": prompt,
        "temperature": temperature
    }
    api_key = <your api key>
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    # print(response.text) 
    response_json = response.json()
    if "choices" in response_json and len(response_json["choices"]) > 0:
        text = response_json["choices"][0]["text"]
        # Remove the triple backticks (```) from the start and end of the text
        text = text.strip("```").strip()
        
        # Check if the text is valid JSON
        try:
            parsed_text = json.loads(text)
            return parsed_text
        except json.JSONDecodeError:
            # If the text is not valid JSON, return it as is
            return text
    else:
        raise ValueError("Unexpected response format from Nugen API")


**This function handles the interaction with the Nugen API:**

1. Makes an HTTP POST request to the API endpoint
2. Includes authentication and request parameters
3. Processes the response, handling both JSON and text formats
4. Returns the processed response from the language model

2. **orchestrator_workflow**

In [11]:
async def orchestrator_workflow(task: str, orchestrator_prompt: str, worker_prompt: str):
    """Use an orchestrator model to break down a task into sub-tasks and then use worker models to generate and return responses."""
    # Use orchestrator model to break the task up into sub-tasks
    orchestrator_response = await run_nugen_llm(orchestrator_prompt.format(task=task))
    
    # Parse orchestrator response 
    analysis = orchestrator_response["analysis"]
    tasks = orchestrator_response["approaches"]
    
    print("\n=== ORCHESTRATOR OUTPUT ===") 
    print(f"\nANALYSIS:\n{analysis}")
    print(f"\nTASKS:\n{json.dumps(tasks, indent=2)}")
    
    # Gather intermediate responses from worker models
    worker_responses = await asyncio.gather(*[
        run_nugen_llm(worker_prompt.format(original_task=task, task_type=task_info["name"], task_description=task_info["description"])) 
        for task_info in tasks
    ])
    
    return tasks, worker_responses

**This is the heart of the orchestrator-worker pattern**:

1. First, it uses the orchestrator model to analyze the task and break it into sub-tasks
2. Prints the analysis and task breakdown for transparency
3. Uses asyncio.gather to run multiple worker models in parallel
4. Each worker generates content for a specific style/approach
5. Returns both the task definitions and worker responses

3. **Main function**

To get started with Nugen LLM models, you'll need an API key. Here's how to mention it in the cookbook:

To access Nugen's LLM capabilities, sign up at [Nugen](https://nugen-platform-frontend.azurewebsites.net/dashboard) 
Obtain your API key from the dashboard. Your API key should be in the format: nugen-xxxxxxxxxxxxxxxxxxxxxx"

In [13]:
async def main():
    task = """Write a product description for a new eco-friendly water bottle. 
    The target_audience is environmentally conscious millennials and key product features are: plastic-free, insulated, lifetime warranty
    """
    tasks, worker_resp = await orchestrator_workflow(task, orchestrator_prompt=ORCHESTRATOR_PROMPT, worker_prompt=WORKER_PROMPT)
    
    for task_info, response in zip(tasks, worker_resp):
        print(f"\n=== WORKER RESULT ({task_info['name']}) ===\n{response}\n")

await main()


=== ORCHESTRATOR OUTPUT ===

ANALYSIS:
The task requires writing a product description for an eco-friendly water bottle targeting environmentally conscious millennials. The key features to highlight are the plastic-free material, insulation, and lifetime warranty. A good product description should not only inform but also persuade and engage the audience. Different approaches can serve different aspects of the task, such as emphasizing technical specifications, creating an emotional connection, or balancing both.

TASKS:
[
  {
    "name": "Formal style",
    "description": "This approach focuses on providing detailed technical specifications of the product, such as the type of insulation used, the material composition, and the warranty terms. It is ideal for readers who prioritize functionality and want to know the nitty-gritty details of the product."
  },
  {
    "name": "Conversational style",
    "description": "This approach involves writing in a friendly and approachable tone, u

This is the entry point that:

1. Defines a sample task (writing a product description)
2. Calls the orchestrator workflow
3. Prints results from each worker, labeled with their approach

**The overall flow works like this**:

1. You provide a task (like writing a product description)
2. The orchestrator analyzes it and creates different approaches (formal, conversational, etc.)
3. Multiple workers then generate content in parallel, each following their assigned style
4. All results are collected and presented together

**This pattern is particularly useful because:**

1. It breaks down complex tasks into manageable pieces
2. Allows for parallel processing of sub-tasks
3. Provides multiple perspectives or approaches to the same problem
4. Uses structured data validation to ensure consistency
5. Handles asynchronous operations efficiently

The code is especially valuable for content generation tasks where you want multiple variations or approaches to the same prompt, all generated efficiently in parallel.